## Part 1: Preprocessing

In [814]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers, regularizers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [815]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [817]:
# Create a list of at least 10 column names to use as X data
sel_columns = ["Age", "BusinessTravel", "Education", "EducationField", "HourlyRate", "JobSatisfaction",\
              "MaritalStatus", "PercentSalaryHike", "PerformanceRating", "YearsSinceLastPromotion"]

# Create X_df using your selected columns
X_df = attrition_df[sel_columns]

# Show the data types for X_df
X_df.dtypes

Age                         int64
BusinessTravel             object
Education                   int64
EducationField             object
HourlyRate                  int64
JobSatisfaction             int64
MaritalStatus              object
PercentSalaryHike           int64
PerformanceRating           int64
YearsSinceLastPromotion     int64
dtype: object

In [818]:
# Create y_df with the Attrition and Department columns
y_attrition_df = attrition_df["Attrition"]
y_department_df = attrition_df["Department"]

In [853]:
print(y_attrition_df.value_counts(normalize=True))

Attrition
No     0.838776
Yes    0.161224
Name: proportion, dtype: float64


In [854]:
print(y_department_df.value_counts(normalize=True))

Department
Research & Development    0.653741
Sales                     0.303401
Human Resources           0.042857
Name: proportion, dtype: float64


In [819]:
# Create a OneHotEncoder for the Attrition column
y_attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
# y_attrition_encoded = y_attrition_encoder.fit_transform(y_attrition_df["Attrition"])
y_attrition_encoded = y_attrition_encoder.fit_transform(y_attrition_df.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data

y_attrition_encoded

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [820]:
# Create a OneHotEncoder for the Department column
y_department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_department_encoded = y_department_encoder.fit_transform(y_department_df.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_encoded 

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [823]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# List of columns that have datatype = object
object_columns = X_df.select_dtypes(include=["object"]).columns

# Instantiate label encoder
le = LabelEncoder()

# Encoding for selected columns
for col in object_columns:
    X_df[col] = le.fit_transform(X_df[col])

#Check data types again
X_df.dtypes

Age                        int64
BusinessTravel             int64
Education                  int64
EducationField             int64
HourlyRate                 int64
JobSatisfaction            int64
MaritalStatus              int64
PercentSalaryHike          int64
PerformanceRating          int64
YearsSinceLastPromotion    int64
dtype: object

In [824]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_attr_train, y_attr_test, y_dept_train, y_dept_test = train_test_split(X_df, y_attrition_encoded, y_department_encoded)

In [825]:
# Create a StandardScaler
scaler = StandardScaler() #.fit(X_train)

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Create, Compile, and Train the Model

In [828]:
X_train_scaled.shape

(1102, 10)

In [829]:
X_test_scaled.shape

(368, 10)

In [830]:
y_attr_train.shape

(1102, 2)

In [831]:
y_attr_test.shape

(368, 2)

In [832]:
y_dept_train.shape

(1102, 3)

In [833]:
y_dept_test.shape

(368, 3)

In [841]:
# Find the number of columns in the X training data
column_nums = (X_train.shape[1],)

# Create the input layer
input_layer = layers.Input(shape=column_nums, name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(32, activation='relu')(input_layer)
shared_layer2 = layers.Dense(16, activation='relu')(shared_layer1)
shared_layer3 = layers.Dense(12, activation='relu')(shared_layer2)

# # To improve accuracy Increase model complexity and add regularization
# input_layer = layers.Input(shape=column_nums, name='input_features')
# x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
# x = layers.Dropout(0.3)(x)

In [842]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = layers.Dense(8, activation='relu', name='attr_hidden')(shared_layer3)

# Create the output layer
attr_output = layers.Dense(2, activation='softmax',name='attr_output')(attr_hidden)


In [843]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(8, activation='relu', name='dept_hidden')(shared_layer3)

# Create the output layer
dept_output = layers.Dense(3, activation='softmax', name='dept_output')(dept_hidden)


In [844]:
# Create the model
# model = Model(inputs=input_layer, outputs=[attr_output, dept_output])
model = Model(inputs=input_layer, 
              outputs=[attr_output, dept_output],
              name='predict_attrition_model')

# Compile the model
model.compile(optimizer='adam',
              loss={'attr_output': 'categorical_crossentropy',
                    'dept_output': 'categorical_crossentropy'},
              loss_weights={'attr_output': 1.0, 'dept_output': 2.0, },
              metrics={'attr_output': 'accuracy',
                       'dept_output': 'accuracy'}
              )

# Summarize the model
model.summary()

Model: "predict_attrition_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_70 (Dense)    │ (None, 32)        │        352 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_71 (Dense)    │ (None, 16)        │        528 │ dense_70[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_72 (Dense)    │ (None, 12)        │        204 │ dense_71[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_hidden (Dense) │ (None, 8)         │        104 │ dense_72[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_hidden (Dense) │ (None, 8)         │        104 │ dense_72[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_output (Dense) │ (None, 2)         │         18 │ attr_hidden[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         27 │ dept_hidden[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,337 (5.22 KB)

 Trainable params: 1,337 (5.22 KB)

 Non-trainable params: 0 (0.00 B)

In [845]:
# Train the model
model.fit(
    X_train_scaled,
    {'attr_output': y_attr_train, 'dept_output': y_dept_train},
    validation_data=(X_test, {'attr_output': y_attr_test, 'dept_output': y_dept_test}),
    epochs=15,
    batch_size=5,
    validation_split=0.2,
    verbose=1
)

Epoch 1/15
221/221 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - attr_output_accuracy: 0.7688 - dept_output_accuracy: 0.4606 - loss: 2.5275 - val_attr_output_accuracy: 0.8261 - val_dept_output_accuracy: 0.6332 - val_loss: 22.5111
Epoch 2/15
221/221 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - attr_output_accuracy: 0.8234 - dept_output_accuracy: 0.6565 - loss: 2.0136 - val_attr_output_accuracy: 0.8261 - val_dept_output_accuracy: 0.6332 - val_loss: 24.0732
Epoch 3/15
221/221 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - attr_output_accuracy: 0.8668 - dept_output_accuracy: 0.6455 - loss: 1.9067 - val_attr_output_accuracy: 0.8261 - val_dept_output_accuracy: 0.6332 - val_loss: 21.5130
Epoch 4/15
221/221 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - attr_output_accuracy: 0.8517 - dept_output_accuracy: 0.6432 - loss: 1.9531 - val_attr_output_accuracy: 0.8261 - val_dept_output_accuracy: 0.6332 - val_loss: 24.4727
Epoch 5/15
221/221 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - attr_output_accuracy: 0.8449 - dept_output_accuracy: 0.6559 - 

In [846]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'attr_output': y_attr_test, 'dept_output': y_dept_test}, verbose=1)
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - attr_output_accuracy: 0.8149 - dept_output_accuracy: 0.6223 - loss: 2.1780


[2.005610704421997, 0.8369565010070801, 0.633152186870575]

In [847]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {test_results[1]:.2f}")
print(f"Department Accuracy: {test_results[2]:.2f}")

Attrition Accuracy: 0.84
Department Accuracy: 0.63


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric to use for this example since data seems to be imbalnced. Accuracy can be misleading for imbalanced datasets, as it may give a high score even if the model performs poorly on minority classes. 
2. Softmax activation function is used for ouput layers. The softmax activation function is commonly used in the output layer of a neural network when dealing with multi-class classification problems. In this example Attrition and Department both have mulitple possible outcomes. Softmax allows the network to output interpretable probability distributions across multiple classes. 
3. Following steps might be helpful in model's perfromance: Select additional features for model
   - Select additional features
   - Select features that have greater impact on outcomes (subject matter expertise is necessary)
   - Increase complexity of the model by increasing number of layers and/or number neurons.
   - Experiment with different learning rates, batch sizes, and optimizers.
   - If possible, generate synthetic data to increase the size of dataset.